# Tweets Scraper

### Importing libraries

In [1]:
import tweepy
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

### Credentials to access API

In [2]:
ACCESS_TOKEN = "1466748284383346695-KgkEL8Ry6tj2NF8Fclz3q3AXAmyUTy"
ACCESS_SECRET = '1psrdL331gCnuHK0ocWKcQhOD22aMcCDY0LFlgXoS21aJ'
CONSUMER_KEY = 'ACrCHwYoxkxOjkbzBNcFQSULg'
CONSUMER_SECRET = 'i75oC3KIiO2w52KcuG9v7SfI2EZL2AbijwhPpGZn1hbxutmF8a'
BEARER_TOKEN =  'AAAAAAAAAAAAAAAAAAAAAO56WgEAAAAA%2FqMC%2B08buZkt5qdMN%2BhYPBCJkfc%3D2fNlcsYgOEFg84jRzGFf7VD5QMQxQKSQp8mEiVqwVreu3sQIfQ'

In [3]:
# Client object to access API
client = tweepy.Client(BEARER_TOKEN)

### Setting Query Parameters

In [86]:
# Query parameters

query = 'omicron OR #omicron'
start_time = '2022-01-07T01:40:01+05:30'
end_time = '2022-01-07T02:59:59+05:30'
next_token = None

max_results = 100
expansions=['attachments.poll_ids', 'attachments.media_keys', 'author_id', 'entities.mentions.username', 'geo.place_id', 'in_reply_to_user_id', 'referenced_tweets.id', 'referenced_tweets.id.author_id']
tweet_fields = ['author_id', 'attachments', 'context_annotations', 'conversation_id', 'created_at', 'entities', 'geo', 'id', 'in_reply_to_user_id','lang', 'public_metrics', 'possibly_sensitive', 'referenced_tweets', 'reply_settings', 'source', 'text', 'withheld']
place_fields = ['contained_within', 'country', 'country_code', 'full_name', 'geo', 'id', 'name', 'place_type']
user_fields = ['created_at', 'description', 'entities', 'id', 'location', 'name', 'pinned_tweet_id', 'profile_image_url', 'protected', 'public_metrics', 'url', 'username', 'verified', 'withheld']

In [78]:
def current_date_time():
    import datetime
    n = datetime.datetime.now(datetime.timezone.utc)
    return n.isoformat()

### Dataframes to store results


In [79]:
# Creating dataframe to store results
columns = ['id', 'text', 'author_id', 'conversation_id', 'in_reply_to_user_id', 'context_annotations', 'lang', 'geo', 
           'source', 'entities', 'reply_settings', 'referenced_tweet_id', 'referenced_tweet_type', 'created_at', 
           'possibly_sensitive', 'public_metrics', 'attachments', 'referenced_tweet_text', 'next_token']
dataset = pd.DataFrame(columns = columns)

# To store user information
user_data = pd.DataFrame(columns = ['id', 'name', 'username'])

# To store

### Making Request to API

In [117]:
# Making query to API
# First query is made without next_token and with timestamp

while(True):
    row_count = dataset.shape[0]
    row_count_2 = user_data.shape[0]
    tweets = client.search_recent_tweets(query=query,
                                                 start_time=start_time,
                                                 end_time=end_time,
                                                 tweet_fields=tweet_fields, 
                                                 expansions=expansions, 
                                                 place_fields=place_fields, 
                                                 user_fields = user_fields,
                                                 max_results=max_results, 
                                                 next_token=next_token)
    count = 0 # Count of returned tweets
    for tweet in tweets.data:
        new_row=[tweet.id,
                 tweet.text,
                 tweet.author_id,
                 tweet.conversation_id,
                 tweet.in_reply_to_user_id,
                 tweet.context_annotations,
                 tweet.lang,
                 tweet.geo,
                 tweet.source,
                 tweet.entities,
                 tweet.reply_settings,
                 tweet.referenced_tweets[0]['id'] if tweet.referenced_tweets!=None else None,
                 tweet.referenced_tweets[0]['type'] if tweet.referenced_tweets!=None else None,
                 tweet.created_at,
                 tweet.possibly_sensitive,
                 tweet.public_metrics,
                 tweet.attachments
                 ]

        # Adding referenced tweet text
        if tweet.referenced_tweets != None:
            ref_id = int(tweet.referenced_tweets[0]['id'])
            flag = 0
            for i in tweets.includes['tweets']:
                if i['id'] == ref_id: 
                    new_row.append(i['text'])
                    flag = 1
                    break
            if flag == 0:
                new_row.append(None)
        else: 
            new_row.append(None)

        # Note: Referenced tweet author details are in mentions

    #     if 'places' in tweets.includes:
    #         new_row.append(tweets.includes['places'][count]['text'])

        new_row.append(tweets.meta['next_token'])
        dataset.loc[row_count] = new_row
        row_count = row_count + 1
        count = count + 1

    for i in range(len(tweets.includes['users'])):
        user_data.loc[row_count_2] = [tweets.includes['users'][i]['id'], 
                                      tweets.includes['users'][i]['name'], 
                                      tweets.includes['users'][i]['username']]
        row_count_2 += 1



    next_token = tweets.meta['next_token']
    start_time = None
    end_time = None
    print(dataset.shape[0])

14345
14445
14543
14642
14740
14840
14937
15033
15131
15231
15330
15428
15527
15627
15725
15824
15922
16022
16122
16220
16320
16420
16518
16615
16714
16813
16911
17011
17111
17209
17308
17408
17506
17603
17701
17800
17899
17997
18097
18195
18292
18392
18492
18592
18688
18786
18884
18984


KeyboardInterrupt: 

In [116]:
def dup_count(df):
    return df.id.duplicated().value_counts()

dup_count(dataset)

False    14158
True        88
Name: id, dtype: int64

### Saving dataframe

In [110]:
def save_df(name):
    dataset.to_csv(name)

In [118]:
# save_df('dataset_v2.csv')

### Merging dataframes

In [109]:
def merge(path):
    old_df = pd.read_csv(path)
    old_df.drop(old_df.columns[0], axis = 1, inplace = True)
    dataset = pd.concat([old_df, dataset], axis=0)

### Experimenting

In [ ]:
for tweet in tweets.data:
    tweet.referenced_tweets[0]['id']

In [ ]:
search_id = '1479792484804960257'
for i in range(len(tweets.includes['tweets'])):
    print(tweets.includes['tweets'][i]['id'])
    if tweets.includes['tweets'][i]['id'] == search_id:
        print('found')

In [ ]:
mentions_ids = {}
count = 0
for tweet in tweets.data:
    if 'mentions' in tweet.entities:
        for i in tweet.entities['mentions']:
            print(i)
    else:
        print('no')